In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from IPython.display import display

from sklearn.datasets import load_breast_cancer
#plt.style.use('default')

import shap
print (shap.__version__)

0.48.0


In [46]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns = data.feature_names)
y = data.target

# Модель RF
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify=y)
forest = RandomForestClassifier(n_estimators = 500, random_state = 42)
forest.fit(X_train, y_train)

# предсказания train
y_pred_train = forest.predict(X_train)
y_pred_proba_train = forest.predict_proba(X_train)
y_pred_proba_train.mean(axis=0)

array([0.37511737, 0.62488263])

In [75]:
# предсказания test
y_pred_test = forest.predict(X_test)
y_pred_proba_test = forest.predict_proba(X_test)
y_pred_proba_test.mean(axis=0)

array([0.38131469, 0.61868531])

In [103]:
# возьмем из test "случайно" строку 1
df = pd.DataFrame(X_test.iloc[1]).T
display(df)
first =  forest.predict_proba(df)
first

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
408,17.99,20.66,117.8,991.7,0.1036,0.1304,0.1201,0.08824,0.1992,0.06069,...,21.08,25.41,138.1,1349.0,0.1482,0.3735,0.3301,0.1974,0.306,0.08503


array([[1., 0.]])

In [81]:
# возьмем из test "случайно" строку k
k = 3
display(pd.DataFrame(X_test.iloc[k]).T)
display(forest.predict_proba(df))
df['worst area'] = X_test['worst area'].iloc[k]

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
518,12.88,18.22,84.45,493.1,0.1218,0.1661,0.04825,0.05303,0.1709,0.07253,...,15.05,24.37,99.31,674.7,0.1456,0.2961,0.1246,0.1096,0.2582,0.08893


array([[0.85, 0.15]])

In [105]:
# пока найдем сдвиг только для по одному признаку 'worst area' без добавления остальных признаков в расчет вектора Шепли
delta = np.array([])
for i in range(3,100):
     df['worst area'] = X_test['worst area'].iloc[i]
     delta = np.append(delta, forest.predict_proba(df)[0][1])
delta

array([0.15 , 0.104, 0.   , 0.112, 0.   , 0.   , 0.162, 0.038, 0.162,
       0.   , 0.   , 0.162, 0.17 , 0.   , 0.142, 0.17 , 0.16 , 0.   ,
       0.   , 0.15 , 0.17 , 0.17 , 0.16 , 0.17 , 0.17 , 0.12 , 0.   ,
       0.166, 0.156, 0.14 , 0.154, 0.162, 0.162, 0.17 , 0.156, 0.   ,
       0.17 , 0.   , 0.17 , 0.   , 0.048, 0.   , 0.17 , 0.   , 0.15 ,
       0.   , 0.162, 0.15 , 0.162, 0.17 , 0.   , 0.098, 0.054, 0.162,
       0.   , 0.   , 0.   , 0.158, 0.   , 0.162, 0.   , 0.156, 0.17 ,
       0.   , 0.04 , 0.104, 0.17 , 0.162, 0.166, 0.162, 0.162, 0.17 ,
       0.   , 0.   , 0.   , 0.   , 0.17 , 0.162, 0.17 , 0.156, 0.162,
       0.   , 0.156, 0.   , 0.17 , 0.104, 0.17 , 0.158, 0.   , 0.   ,
       0.17 , 0.   , 0.   , 0.   , 0.16 , 0.014, 0.17 ])

In [106]:
(first[0][1]-delta).mean()

-0.0974020618556701